# 1. Install Dependencies and Setup

In [ ]:
%pip install tensorflow opencv-python matplotlib

In [ ]:
%pip list

In [ ]:
import tensorflow as tf
import os

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')

for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
tf.config.list_physical_devices('GPU')

# 2. Remove dodgy images

In [4]:
import cv2
import imghdr

In [5]:
data_dir = '../data/raw'

In [4]:
image_exts = ['jpg', 'jpeg', 'bmp', 'png']

In [5]:
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

# 3. Load Data

In [6]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('../data/raw', batch_size=16, image_size=(256, 256), color_mode='rgb', shuffle=True)

In [8]:
data_iterator = data.as_numpy_iterator()

In [9]:
batch = data_iterator.next()

In [ ]:
print(batch)

In [ ]:
batch[1]

In [ ]:
fig, ax = plt.subplots(ncols=4, nrows=6, figsize=(20, 20))
for idx, img in enumerate(batch[0][:24]): 
    ax.flat[idx].imshow(img.astype(int))
    ax.flat[idx].title.set_text(batch[1][idx])


# 4. Scale Data

In [10]:
data = data.map(lambda x, y: (x / 255.0, y))

In [ ]:
scaled_iterator = data.as_numpy_iterator()
batch = scaled_iterator.next()
batch[0].max()


In [ ]:
fig, ax = plt.subplots(ncols=4, nrows=6, figsize=(20, 20))
for idx, img in enumerate(batch[0][:24]): 
    ax.flat[idx].imshow(img)
    ax.flat[idx].title.set_text(batch[1][idx])

# 5. Split Data

In [ ]:
data.shape()

In [12]:
train_size = int(len(data)*0.75)
val_size = int(len(data)*0.17)
test_size = int(len(data)*0.08)

In [ ]:
train_size

In [ ]:
val_size

In [ ]:
test_size

In [13]:
train_data = data.take(train_size)
val_data = data.skip(train_size).take(val_size)
test_data = data.skip(train_size + val_size).take(test_size)

In [ ]:
len(test_data)

# 6. Build Deep Learning Model

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.regularizers import l2

In [ ]:
model = Sequential([
        # First Convolutional Block
        Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3), 
               kernel_regularizer=l2(0.01)),
        MaxPooling2D((2, 2)),
        
        # Second Convolutional Block
        Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
        MaxPooling2D((2, 2)),
        
        # Third Convolutional Block
        Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
        MaxPooling2D((2, 2)),
        
        # Fourth Convolutional Block
        Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
        MaxPooling2D((2, 2)),
        
        # Flatten the output and add dense layers
        Flatten(),
        Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
        Dropout(0.5),
        Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
        Dropout(0.3),
        Dense(24, activation='softmax')
    ])
    
    

In [16]:
model = Sequential()

In [ ]:
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [16]:
# Compile the model
model.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
model.summary()

# 7. Train

In [15]:
logdir = r'../data/logs'

In [16]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(train_data, validation_data=val_data, epochs=20, callbacks=[tensorboard_callback])

In [22]:
training_config = {
    'epochs': 20,
    'callbacks': [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=3,
            min_lr=1e-6
        ),
        tensorboard_callback
    ]
}

In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=training_config['epochs'],
    callbacks=training_config['callbacks']
)

# 8. Testing Training


In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [18]:
logdir = r'../data/logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
# Much smaller L2 regularization
reg_factor = 0.005

model = Sequential([
    # First Convolutional Block - reduced initial filters
    Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(256, 256, 3),
            kernel_regularizer=l2(reg_factor)),
    BatchNormalization(),
    Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(reg_factor)),
        BatchNormalization(),
    MaxPooling2D((2, 2)),
    
    # Second Convolutional Block
    Conv2D(64, (3, 3), padding='same', activation='relu',
            kernel_regularizer=l2(reg_factor)),
    BatchNormalization(),
    Conv2D(64, (3, 3), padding='same', activation='relu',
            kernel_regularizer=l2(reg_factor)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    
    # Third Convolutional Block
    Conv2D(128, (3, 3), padding='same', activation='relu',
            kernel_regularizer=l2(reg_factor)),
    BatchNormalization(),
    Conv2D(128, (3, 3), padding='same', activation='relu',
            kernel_regularizer=l2(reg_factor)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    
    # Fourth Convolutional Block
    Conv2D(256, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(reg_factor)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    
    # Flatten the output and add dense layers
    Flatten(),
    Dense(256, activation='relu', kernel_regularizer=l2(reg_factor)),
    BatchNormalization(),
    Dropout(0.4),  # Slightly reduced dropout for longer training
    Dense(128, activation='relu', kernel_regularizer=l2(reg_factor)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(24, activation='softmax')
])

In [20]:
optimizer = Adam(learning_rate=0.0001)

In [21]:
model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

In [24]:
# Data augmentation for better generalization during longer training
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomBrightness(0.2),
    tf.keras.layers.RandomContrast(0.2),
])

In [23]:
# Enhanced training configuration for longer training
training_config = {
    'epochs': 15,  # Increased epochs
    'batch_size': 16,
    'callbacks': [
        EarlyStopping(
            monitor='val_loss',
            patience=15,  # Increased patience
            restore_best_weights=True,
            min_delta=0.0001  # Smaller improvement threshold
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=8,  # Increased patience
            min_lr=1e-7,  # Lower minimum learning rate
            min_delta=0.0001
        ),
        tensorboard_callback
    ]
}

In [25]:
augmented_train_data = train_data.map(lambda x, y: (data_augmentation(x,training=True), y))

In [ ]:
print("Sample batch shape:", next(iter(train_data))[0].shape)
print("Value range:", tf.reduce_min(next(iter(train_data))[0]).numpy(), 
      tf.reduce_max(next(iter(train_data))[0]).numpy())

In [27]:
train_data = train_data.map(lambda x, y: (tf.clip_by_value(x, 0, 255.) / 255., y))
val_data = val_data.map(lambda x, y: (tf.clip_by_value(x, 0, 255.) / 255., y))


In [ ]:
history = model.fit(
    augmented_train_data,
    validation_data=val_data,
    epochs=training_config['epochs'],
    batch_size=training_config['batch_size'],
    callbacks=training_config['callbacks']
)

In [ ]:
fig = plt.figure()
plt.plot(history.history['loss'], color='teal', label='loss')
plt.plot(history.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(history.history['accuracy'], color='teal', label='accuracy')
plt.plot(history.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [29]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [30]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
for batch in test_data.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(pre.result(), re.result(), acc.result())

In [ ]:
import tensorflow as tf
import cv2
from matplotlib import pyplot as plt

img = cv2.imread('../test_image.jpg')
if img is not None:
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img_rgb)
    plt.show()
else:
    print("Error: Image not found or unable to load.")


In [ ]:
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
yhat = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
yhat

In [ ]:
if (yhat >= 14.0) and (yhat <= 15.0):
    print("plus 2 red")
else:
    print("not plus 2 red")